Install if needed

In [1]:
!pip install --upgrade google-auth-oauthlib

  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.1
    Uninstalling google-auth-oauthlib-0.4.1:
      Successfully uninstalled google-auth-oauthlib-0.4.1


**Import the required libraries**

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

import numpy as np
import pandas as pd

from dateutil.relativedelta import relativedelta


<br/><br/>**Import the Alarms dataset from Google Cloud Storage**

**Filtering the Alarms dataset by time and inhibit or not**

**the variables Days_before represent the minimun of dates between alarms**

**the variables Days_From_beginning represent the days between the first day and each alarms**

In [6]:
path = "gs://table-cloud-lab-ctrlsenales-bucket/PIC_TRACK_ALARMS.csv"
df_Alarms = pd.read_csv(path)

def Alarm_Detection( df_Alarms ):
    import pandas as pd

    df_Alarms = df_Alarms[df_Alarms['ID'] == 'Alarma Alta Temperatura en servicio']

    df_Alarms = df_Alarms[['IDEquipo']+['TS']+['Type']+['Equipo']+['Centro']+['IDCentro']+['IDZona']]



    df_Alarms = df_Alarms[df_Alarms['Equipo'].str.contains("Mural Carne|Mural Pescado")]

    df_Alarms.columns = ['ELEMENT', 'TS','ALTA_TEMPERATURA', 'Equipo', 'Centro', 'IDCentro', 'IDZona']




    #tranform the column TS to a datetime Object
    df_Alarms['TS'] = pd.to_datetime(df_Alarms['TS'], infer_datetime_format=True)

    first = df_Alarms['TS'].min()
    #Sort the values by the time
    df_Alarms = df_Alarms.sort_values(['ELEMENT','TS'], ascending=[ True, True]).reset_index(drop=True)

    #cols=list(df_Alarms.columns)

    #df_Alarms = df_Alarms[cols[0:2]+[cols[4]]]


    df_Alarms = df_Alarms.dropna().reset_index(drop=True)

    #papa pa paaaannnnnnnnnnnnnnnnnnnnnnnnnn
    #Check if the Begin was 'InhibitNotif' before the event occurred
    #Create a "new" column to indicate if it is an alarm or an alarm inhibited by user (on these cases it will have TRUE)

    for i, row  in df_Alarms.iterrows():
        if (df_Alarms.loc[i,'ALTA_TEMPERATURA']) =='Begin' and  (df_Alarms.loc[i-1,'ALTA_TEMPERATURA'])!='InhibitNotif':
            df_Alarms.at[i,'new'] = 'True'
        elif (df_Alarms.loc[i,'ALTA_TEMPERATURA']) =='Begin' and  (df_Alarms.loc[i-1,'ALTA_TEMPERATURA'])=='InhibitNotif':
            df_Alarms.at[i,'new'] = 'True but InhibitNotif by user'
        else:
            df_Alarms.at[i,'new'] = 'False'      

    df_InhibitAlarms = df_Alarms[df_Alarms['new'] == 'True but InhibitNotif by user']
    print('\n These are the Alarms Inhibit by an user before it occurred: \n')
    print(df_InhibitAlarms.loc[:,['TS','ELEMENT', 'new']].reset_index(drop=True))
    df_Alarms = df_Alarms[(df_Alarms['new'] == 'True') | (df_Alarms['new'] == 'True but InhibitNotif by user')]

    #df_Alarms = df_Alarms.drop('new', axis=1).reset_index(drop=True)
    df_Alarms = df_Alarms.sort_values(['ELEMENT','TS'], ascending=[ True, True]).reset_index(drop=True)

    #Check is the prior event occurred in a especific time
    '''

    for i in range(len(df_Alarms)):
        if i == 0: continue
        timedelta =((df_Alarms.loc[i,'TS'])-(df_Alarms.loc[i-1,'TS'])).days
        print("Es el mismo elemento que el anterior ", (df_Alarms.loc[i,'ELEMENT']) == (df_Alarms.loc[i-1,'ELEMENT']))
        print("Time Delta = ",timedelta)
    '''

    #Add a new column "new2" to indicate if there isn't any previous alarm in Days_before days
    #We discard the alarms that had another alarm in the Days_before previous days.
    
    for i, row  in df_Alarms.iterrows():
        Days_before = 11
        if i == 0:
            df_Alarms.at[i,'new2'] = 'True'
        elif (df_Alarms.loc[i,'ELEMENT']) == (df_Alarms.loc[i-1,'ELEMENT']):
            if ((df_Alarms.loc[i,'TS'])-(df_Alarms.loc[i-1,'TS'])).days >= Days_before:
                df_Alarms.at[i,'new2'] = 'True'
            else:
                df_Alarms.at[i,'new2'] = 'Too Short' 
        else:
            df_Alarms.at[i,'new2'] = 'True'

    df_Tooshort = df_Alarms[df_Alarms['new2'] == 'Too Short' ]
    print('\n These are the Alarms too close to each other: \n')

    print(df_Tooshort.loc[:,['TS','ELEMENT', 'new']].reset_index(drop=True))
    df_Alarms = df_Alarms[df_Alarms['new'] == 'True']
    df_Alarms = df_Alarms[df_Alarms['new2'] == 'True']

    df_Alarms = df_Alarms.drop('new', axis=1).reset_index(drop=True)
    df_Alarms = df_Alarms.drop('new2', axis=1).reset_index(drop=True)
    df_Alarms = df_Alarms.sort_values(['ELEMENT','TS'], ascending=[ True, True]).reset_index(drop=True)

    #to check if the timedelta between the first event and the alarm is more than 15days 


    for i, row  in df_Alarms.iterrows():
        Days_From_beginning = 15
        if ((df_Alarms.loc[i,'TS'])-first).days >= Days_From_beginning:
            df_Alarms.at[i,'new'] = 'True'
        else:
            df_Alarms.at[i,'new'] = 'Not enough data' 

    df_Not_enough = df_Alarms[df_Alarms['new'] == 'Not enough data' ]

    print('\n These are the Alarms with too little data: \n')
    print(df_Not_enough.loc[:,['TS','ELEMENT', 'new']].reset_index(drop=True))

    df_Alarms = df_Alarms[df_Alarms['new'] == 'True']
    df_Alarms = df_Alarms.drop('new', axis=1).reset_index(drop=True)
    df_Alarms = df_Alarms.sort_values(['TS','ELEMENT'], ascending=[ True, True]).reset_index(drop=True)
    
    
    df_Alarms = df_Alarms.sort_values(['ELEMENT', 'TS'], ascending=[True, True]).reset_index(drop=True)
    
    #keep only the colums element, timestand and alarm
    df_Alarms = df_Alarms[['ELEMENT']+['TS']+['ALTA_TEMPERATURA']]
    
    
    #create a new row with a ten days prior the begin date
    for i in range(len(df_Alarms)):
        timeframe = df_Alarms.loc[i, 'TS'] - relativedelta(days=10)
        df_Alarms = df_Alarms.append({'TS':timeframe,'ELEMENT':(df_Alarms.loc[i, 'ELEMENT']),'ALTA_TEMPERATURA':'Start'}, ignore_index=True)
    
    df_Alarms = df_Alarms.sort_values(['ELEMENT', 'TS'], ascending=[True, True]).reset_index(drop=True)
    
    #create new column called cycle_number to difference the different cycles
    df_Alarms['cycle_number'] = df_Alarms['ALTA_TEMPERATURA'].map({'Start':0, 'Begin':1}).diff().lt(0).cumsum() + 1 
    df_Alarms= df_Alarms.drop(['ALTA_TEMPERATURA'], axis=1).reset_index(drop=True)

    
    print('\n This are the final Alarms dataset \n')
    print(df_Alarms)
    return(df_Alarms)


#The function that apply the filter modifications to the dataset
df_Alarms = Alarm_Detection(df_Alarms)


 These are the Alarms Inhibit by an user before it occurred: 

                     TS  ELEMENT                            new
0   2019-05-23 12:02:00    29102  True but InhibitNotif by user
1   2019-05-23 12:06:00    29103  True but InhibitNotif by user
2   2019-05-23 12:08:00    29104  True but InhibitNotif by user
3   2019-05-23 12:01:00    29115  True but InhibitNotif by user
4   2019-05-23 12:23:00    29116  True but InhibitNotif by user
..                  ...      ...                            ...
371 2020-02-01 12:11:00   141044  True but InhibitNotif by user
372 2020-02-04 12:13:00   141044  True but InhibitNotif by user
373 2020-01-08 01:50:00   141970  True but InhibitNotif by user
374 2020-01-08 02:04:00   141982  True but InhibitNotif by user
375 2020-05-14 13:26:00   155176  True but InhibitNotif by user

[376 rows x 3 columns]

 These are the Alarms too close to each other: 

                     TS  ELEMENT                            new
0   2019-06-05 10:52:00    302

In [4]:
elements_alarms= df_Alarms['ELEMENT'].unique()
cycles_alarms= df_Alarms['cycle_number'].unique()
print("Count of elements in alarms: ",len(elements_alarms))
print("Count of cycles in alarms: ",len(cycles_alarms))


Count of elements in alarms:  139
Count of cycles in alarms:  158


Create the client for the services account needed to access the bigquery storage in project "madonna-cloud-labpri-ctrlsenales"

In [5]:
# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "Credentials.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

**Import the Data from bigquery**

In [6]:
query = """
SELECT
  TS,
  ELEMENT,
  TAG_SONDA_PB1,
  TAG_SONDA_PB2,
  TAG_PRESION_SATURACION,
  TAG_TEMP_ASPIRACION,
  TAG_RECALENT_VALVULA,
  TAG_APERT_VALVULA,
  TAG_EQUIPO_STANDBY,
  TAG_PETICION_FRIO,
  TAG_COMUNICA,
  TAG_DESCARCHE,
  TAG_ALARMA,
FROM
  `table-cloud-labpri-ctrlsenales.DAN_PIC_Europe.PIC_TRACK_VARIABLES_PREP` 
 
        """
#client = bigquery.Client(location="EU", project="table-cloud-labpri-ctrlsenales")


query_job = client.query(query,location="EU") 

df_Variables = query_job.to_dataframe()
df_Variables.head()

,TS,ELEMENT,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_EQUIPO_STANDBY,TAG_PETICION_FRIO,TAG_COMUNICA,TAG_DESCARCHE,TAG_ALARMA
0,2019-06-13 10:00:00+00:00,61407,NaN,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 19:49:00+00:00,61407,NaN,NaN,NaN,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-10-16 14:11:24+00:00,61407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,2019-08-04 17:12:43+00:00,61407,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-06-22 13:55:21+00:00,61407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [7]:
elements_Variables= df_Variables['ELEMENT'].unique()
print("Count of elements in alarms: ",len(elements_Variables))


Count of elements in alarms:  149


In [8]:
print("this are the excluded elements from the variable with no alarms: " ,list(set(elements_Variables) - set(elements_alarms)))
print("Number of elements excuded: " ,len(list(set(elements_Variables) - set(elements_alarms))))

this are the excluded elements from the variable with no alarms:  [139456, 31616, 31617, 121641, 109961, 134349, 31183, 64371, 87766, 51735, 51736, 51737, 51738, 31615]
Number of elements excuded:  14


Check the columns types

In [9]:
df_Variables.dtypes

TS                        datetime64[ns, UTC]
ELEMENT                                 int64
TAG_SONDA_PB1                         float64
TAG_SONDA_PB2                         float64
TAG_PRESION_SATURACION                float64
TAG_TEMP_ASPIRACION                   float64
TAG_RECALENT_VALVULA                  float64
TAG_APERT_VALVULA                     float64
TAG_EQUIPO_STANDBY                    float64
TAG_PETICION_FRIO                     float64
TAG_COMUNICA                          float64
TAG_DESCARCHE                         float64
TAG_ALARMA                            float64
dtype: object

Count the values of each column

In [10]:
df_Variables.count()

TS                        251981194
ELEMENT                   251981194
TAG_SONDA_PB1              32088897
TAG_SONDA_PB2              49595855
TAG_PRESION_SATURACION      7877515
TAG_TEMP_ASPIRACION        36113613
TAG_RECALENT_VALVULA       59905952
TAG_APERT_VALVULA          38636691
TAG_EQUIPO_STANDBY               26
TAG_PETICION_FRIO          24567466
TAG_COMUNICA                1761716
TAG_DESCARCHE               1000361
TAG_ALARMA                    62099
dtype: int64

Convert the timestamp to a format without UTC location

In [11]:
df_Variables['TS'] = df_Variables['TS'].dt.tz_localize(None)

Describe the variables 

In [12]:
df_Variables.describe()

,ELEMENT,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_EQUIPO_STANDBY,TAG_PETICION_FRIO,TAG_COMUNICA,TAG_DESCARCHE,TAG_ALARMA
count,2.519812e+08,3.208890e+07,4.959586e+07,7.877515e+06,3.611361e+07,5.990595e+07,3.863669e+07,26.000000,24567466.0,1.761716e+06,1.000361e+06,62099.000000
mean,7.328044e+04,1.287077e+00,-1.767519e+00,4.762160e+00,2.012977e+00,1.218265e+01,5.252416e+01,0.500000,0.5,9.513633e-01,4.984141e-01,0.500024
std,2.728419e+04,2.254337e+00,2.899452e+00,6.696564e+00,4.252219e+00,7.634655e+00,4.183083e+01,0.509902,0.5,2.151073e-01,4.999977e-01,0.500004
min,2.911500e+04,-3.700000e+00,-5.240000e+01,-1.000000e+00,-4.640000e+01,-3.265800e+03,0.000000e+00,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000
25%,5.173500e+04,-1.000000e-01,-3.300000e+00,2.100000e+00,-2.000000e-01,7.500000e+00,0.000000e+00,0.000000,0.0,1.000000e+00,0.000000e+00,0.000000
50%,7.472600e+04,8.000000e-01,-1.800000e+00,3.000000e+00,1.800000e+00,1.070000e+01,6.490000e+01,0.500000,0.0,1.000000e+00,0.000000e+00,1.000000
75%,9.220100e+04,1.500000e+00,-4.000000e-01,3.800000e+00,3.800000e+00,1.470000e+01,9.470000e+01,1.000000,1.0,1.000000e+00,1.000000e+00,1.000000
max,1.537050e+05,2.560000e+01,1.124000e+02,3.630000e+01,3.276700e+03,1.266000e+02,1.000000e+02,1.000000,1.0,1.000000e+00,1.000000e+00,1.000000


Drop the columns ['TAG_EQUIPO_STANDBY', 'TAG_COMUNICA','TAG_ALARMA']

In [13]:
df_Variables = df_Variables.sort_values(['ELEMENT', 'TS'], ascending=[True, True]).reset_index(drop=True)

In [14]:
df_Variables

,TS,ELEMENT,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_EQUIPO_STANDBY,TAG_PETICION_FRIO,TAG_COMUNICA,TAG_DESCARCHE,TAG_ALARMA
0,2018-12-14 00:00:21,29115,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,NaN
1,2018-12-14 00:00:21,29115,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-12-14 00:00:21,29115,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-12-14 00:01:21,29115,NaN,NaN,NaN,NaN,9.6,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-12-14 00:02:20,29115,NaN,NaN,NaN,NaN,5.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251981189,2020-06-19 04:54:00,153705,NaN,NaN,NaN,NaN,NaN,78.800003,NaN,NaN,NaN,NaN,NaN
251981190,2020-06-19 04:55:38,153705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
251981191,2020-06-19 04:56:00,153705,-0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251981192,2020-06-19 04:56:00,153705,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


<br/><br/>**Merge the Variables to keep only the data of the alarms**

In [15]:
pd.options.mode.chained_assignment = None

#df_merge['unit_number'] = df_merge['ALTA_TEMPERATURA'].map({'Start':0, 'Begin':1}).diff().lt(0).cumsum() + 1
#------------------------------------------------------------------------------------------------------------------------
def merging_data(data_telemetria, data_alarmas):
    cycles = data_alarmas['cycle_number'].unique()
    df_timeframe = pd.DataFrame([])
    for i  in range(len(cycles)):
        Alarm_cycle =data_alarmas.loc[data_alarmas['cycle_number']==cycles[i]]
        elemento = Alarm_cycle.iloc[0,0]
        t_max = Alarm_cycle['TS'].max() + relativedelta(days=4)
        t_min = t_max - relativedelta(days=15)
        df_loop = data_telemetria.loc[data_telemetria['ELEMENT']==elemento]
        df_loop = df_loop.loc[(df_loop['TS'] > t_min) & (df_loop['TS'] <= t_max)]
        if df_loop.shape[0] >1:
            df_loop.at[:,'cycle_number'] = (i+1)
            df_timeframe = df_timeframe.append(df_loop).reset_index(drop=True)

     
    return( df_timeframe[['TS']+['ELEMENT']+['TAG_SONDA_PB1']+['TAG_SONDA_PB2']+['TAG_PRESION_SATURACION']+
              ['TAG_TEMP_ASPIRACION']+['TAG_RECALENT_VALVULA']+['TAG_APERT_VALVULA']+['TAG_PETICION_FRIO']+
              ['TAG_DESCARCHE']+['cycle_number']])

df_merge = merging_data(df_Variables, df_Alarms )

In [16]:
def label_Cycle(data):
    cycles = data['cycle_number'].unique()
    df = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop = data.loc[data['cycle_number']==cycles[i]]
        
        df_loop.loc[:,'cycle_number']=(i+1)
     
        df = df.append(df_loop).reset_index(drop=True)
    return(df)
    
df_merge = label_Cycle(df_merge)

In [17]:
cycles = df_merge['cycle_number'].unique()
print(cycles)
print(cycles.shape)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154]
(154,)


In [18]:
df_merge

,TS,ELEMENT,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_PETICION_FRIO,TAG_DESCARCHE,cycle_number
0,2019-02-26 14:49:08,29115,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,1
1,2019-02-26 14:49:08,29115,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,1
2,2019-02-26 14:49:08,29115,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2019-02-26 14:49:08,29115,NaN,NaN,NaN,5.9,NaN,NaN,NaN,NaN,1
4,2019-02-26 14:50:08,29115,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
7325845,2020-05-06 17:12:00,153705,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,154
7325846,2020-05-06 17:12:00,153705,NaN,NaN,NaN,-3.2,NaN,NaN,NaN,NaN,154
7325847,2020-05-06 17:14:00,153705,NaN,NaN,NaN,-0.2,NaN,NaN,NaN,NaN,154
7325848,2020-05-06 17:14:00,153705,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,154


In [19]:
elements_Merge= df_merge['ELEMENT'].unique()
ciclos_Merge= df_merge['cycle_number'].unique()
print("Count of elements in alarms: ",len(elements_Merge))
print("Count of cycles in alarms: ",len(ciclos_Merge))


Count of elements in alarms:  135
Count of cycles in alarms:  154


In [20]:
print("this are the excluded elements from the variable with no alarms: " ,list(set(elements_Variables) - set(elements_Merge)))
print("Number of elements excuded: " ,len(list(set(elements_Variables) - set(elements_Merge))))

this are the excluded elements from the variable with no alarms:  [139456, 31616, 31617, 121641, 109961, 134349, 31183, 64371, 87766, 51735, 51736, 51737, 51738, 31615]
Number of elements excuded:  14


Erase the elements with no values in Peticion de frio that belong to the same location : C4420 Centro Coinnovacion Paterna Valencia

In [21]:

df_merge =  df_merge.loc[df_merge['ELEMENT']!=53050]
df_merge =  df_merge.loc[df_merge['ELEMENT']!=53051]
df_merge =  df_merge.loc[df_merge['ELEMENT']!=53052]
df_merge =  df_merge.loc[df_merge['ELEMENT']!=53053]
df_merge =  df_merge.loc[df_merge['ELEMENT']!=53060]
df_merge =  df_merge.loc[df_merge['ELEMENT']!=53061]


In [22]:
def timedelta_Categorico(dataframe, columnas):
    df = dataframe.loc[:, ['ELEMENT','TS',columnas]].dropna().reset_index(drop=True)
    df = df.groupby(['ELEMENT',pd.Grouper(key='TS', freq='1s')]).mean().reset_index()
    for i, row  in df.iterrows():
        if i == 0:
            df.at[i,'TD_0_{}'.format(columnas)] = np.nan
            df.at[i,'TD_1_{}'.format(columnas)] = np.nan
        elif (df.loc[i,columnas]) ==1:
            df.at[i,'TD_1_{}'.format(columnas)] = ((df.loc[i,'TS'])-(df.loc[i-1,'TS'])).total_seconds()
            df.at[i,'TD_0_{}'.format(columnas)] = np.nan
        elif (df.loc[i,columnas]) ==0:
            df.at[i,'TD_0_{}'.format(columnas)] = ((df.loc[i,'TS'])-(df.loc[i-1,'TS'])).total_seconds()
            df.at[i,'TD_1_{}'.format(columnas)] = np.nan
        else:
            df.at[i,'TD_0_{}'.format(columnas)] = np.nan
            df.at[i,'TD_1_{}'.format(columnas)] = np.nan
            
    df= df.drop([columnas], axis=1).reset_index(drop=True)

    df = df.groupby(['ELEMENT',pd.Grouper(key='TS', freq='1min')]).min().reset_index()
    return df



In [23]:
def new_columns(data):
    cycles = data['cycle_number'].unique()
    df_merge_final = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop =  data.loc[data['cycle_number']==cycles[i]]
        df_loop2 = (df_loop.loc[:, ['ELEMENT','TS','TAG_PETICION_FRIO','TAG_DESCARCHE']].set_index('TS').groupby('ELEMENT').resample('1min').mean().drop(['ELEMENT'], 1).reset_index())
        df_loop2.columns = ['ELEMENT', 'TS','SUM_PETICION_FRIO', 'SUM_DESCARCHE']

        df_loop3 = timedelta_Categorico(df_loop, 'TAG_PETICION_FRIO')
        df_loop4 = timedelta_Categorico(df_loop, 'TAG_DESCARCHE')
        
        df_loop =(df_loop.set_index('TS').groupby('ELEMENT').resample('1min').mean().drop(['ELEMENT'], 1).reset_index())
        df_loop['TAG_PETICION_FRIO'] = df_loop['TAG_PETICION_FRIO'].apply(lambda x: 1 if ( x >= 0.5 and x <= 1) else (0 if ( x < 0.5 and x >= 0) else x))
        df_loop['TAG_DESCARCHE'] = df_loop['TAG_DESCARCHE'].apply(lambda x: 1 if ( x >= 0.5 and x <= 1) else (0 if ( x < 0.5 and x >= 0) else x))
        df_loop_final = df_loop.merge(df_loop2, on=['ELEMENT','TS'], how='left')
        df_loop_final = df_loop_final.merge(df_loop3, on=['ELEMENT','TS'], how='left')
        df_loop_final = df_loop_final.merge(df_loop4, on=['ELEMENT','TS'], how='left')
        df_loop_final =(df_loop_final.set_index('TS').groupby('ELEMENT').resample('1min').mean().drop(['ELEMENT'], 1).reset_index())
        df_loop_final.loc[:,'cycle_number']=(i+1)


        
        df_merge_final = df_merge_final.append(df_loop_final).reset_index(drop=True)
    return df_merge_final
        

df_merge = new_columns(df_merge)
df_merge

,ELEMENT,TS,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_PETICION_FRIO,TAG_DESCARCHE,cycle_number,SUM_PETICION_FRIO,SUM_DESCARCHE,TD_0_TAG_PETICION_FRIO,TD_1_TAG_PETICION_FRIO,TD_0_TAG_DESCARCHE,TD_1_TAG_DESCARCHE
0,29115,2019-02-26 14:49:00,NaN,0.6,NaN,5.9,11.0,100.000000,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
1,29115,2019-02-26 14:50:00,1.9,NaN,NaN,3.9,14.4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2,29115,2019-02-26 14:51:00,NaN,NaN,NaN,NaN,11.5,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
3,29115,2019-02-26 14:52:00,NaN,NaN,NaN,NaN,10.1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4,29115,2019-02-26 14:53:00,NaN,NaN,NaN,5.2,8.2,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140635,153705,2020-05-06 17:10:00,NaN,-1.9,NaN,2.6,NaN,82.900002,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN
3140636,153705,2020-05-06 17:11:00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,148,0.0,NaN,115.0,NaN,NaN,NaN
3140637,153705,2020-05-06 17:12:00,NaN,NaN,NaN,-3.2,1.2,0.000000,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN
3140638,153705,2020-05-06 17:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
elements_Merge2= df_merge['ELEMENT'].unique()
print("Count of elements in alarms: ",len(elements_Merge2))
print("this are the excluded elements from the variable with no alarms: " ,list(set(elements_Merge) - set(elements_Merge2)))


Count of elements in alarms:  129
this are the excluded elements from the variable with no alarms:  [53060, 53061, 53050, 53051, 53052, 53053]


In [25]:
query = """
SELECT
  ID_ELEMENT,
  ID_LOCATION,
  
FROM
  `table-cloud-labpri-ctrlsenales.DAN_PIC_Europe.PIC_ELEMENT_LOCATION` 

  
        """
#client = bigquery.Client(location="EU", project="table-cloud-labpri-ctrlsenales")


query_job = client.query(query,location="EU") 

df_location = query_job.to_dataframe()
df_location.head()

,ID_ELEMENT,ID_LOCATION
0,72333,72204
1,72394,72271
2,72381,72256
3,72382,72256
4,72399,72276


In [26]:
df_merge = df_merge.merge(df_location, left_on='ELEMENT', right_on='ID_ELEMENT',  how='left').drop(['ID_ELEMENT'], 1)
df_merge

,ELEMENT,TS,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_PETICION_FRIO,TAG_DESCARCHE,cycle_number,SUM_PETICION_FRIO,SUM_DESCARCHE,TD_0_TAG_PETICION_FRIO,TD_1_TAG_PETICION_FRIO,TD_0_TAG_DESCARCHE,TD_1_TAG_DESCARCHE,ID_LOCATION
0,29115,2019-02-26 14:49:00,NaN,0.6,NaN,5.9,11.0,100.000000,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,29100
1,29115,2019-02-26 14:50:00,1.9,NaN,NaN,3.9,14.4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,29100
2,29115,2019-02-26 14:51:00,NaN,NaN,NaN,NaN,11.5,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,29100
3,29115,2019-02-26 14:52:00,NaN,NaN,NaN,NaN,10.1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,29100
4,29115,2019-02-26 14:53:00,NaN,NaN,NaN,5.2,8.2,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,29100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140635,153705,2020-05-06 17:10:00,NaN,-1.9,NaN,2.6,NaN,82.900002,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN,65423
3140636,153705,2020-05-06 17:11:00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,148,0.0,NaN,115.0,NaN,NaN,NaN,65423
3140637,153705,2020-05-06 17:12:00,NaN,NaN,NaN,-3.2,1.2,0.000000,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN,65423
3140638,153705,2020-05-06 17:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN,65423


Fill the missing values:\
the Continuous variable will be interpolate\
the Categorical variable will be filled with the previous value

In [27]:
def Interpolate_data(data):
    cycles = data['cycle_number'].unique()
    df = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop = data.loc[data['cycle_number'] == (i+1)]
        df_loop.loc[:,'TAG_SONDA_PB1'] = df_loop.loc[:,'TAG_SONDA_PB1'].interpolate()
        df_loop.loc[:,'TAG_SONDA_PB2'] = df_loop.loc[:,'TAG_SONDA_PB2'].interpolate()
        df_loop.loc[:,'TAG_PRESION_SATURACION'] = df_loop.loc[:,'TAG_PRESION_SATURACION'].interpolate()
        df_loop.loc[:,'TAG_TEMP_ASPIRACION'] = df_loop.loc[:,'TAG_TEMP_ASPIRACION'].interpolate()
        df_loop.loc[:,'TAG_RECALENT_VALVULA'] = df_loop.loc[:,'TAG_RECALENT_VALVULA'].interpolate()
        
        
        #filling the missing values in the binary columns 
        df_loop.loc[:,'TAG_APERT_VALVULA'] = df_loop.loc[:,'TAG_APERT_VALVULA'].fillna(method='ffill')
        df_loop.loc[:,'SUM_PETICION_FRIO'] = df_loop.loc[:,'SUM_PETICION_FRIO'].fillna(method='ffill')
        df_loop.loc[:,'SUM_DESCARCHE'] = df_loop.loc[:,'SUM_DESCARCHE'].fillna(method='ffill')
        df_loop.loc[:,'TAG_PETICION_FRIO'] = df_loop.loc[:,'TAG_PETICION_FRIO'].fillna(method='ffill')
        df_loop.loc[:,'TAG_DESCARCHE'] = df_loop.loc[:,'TAG_DESCARCHE'].fillna(method='ffill')
        df_loop.loc[:,'TD_0_TAG_PETICION_FRIO'] = df_loop.loc[:,'TD_0_TAG_PETICION_FRIO'].fillna(method='ffill')
        df_loop.loc[:,'TD_1_TAG_PETICION_FRIO'] = df_loop.loc[:,'TD_1_TAG_PETICION_FRIO'].fillna(method='ffill')
        df_loop.loc[:,'TD_0_TAG_DESCARCHE'] = df_loop.loc[:,'TD_0_TAG_DESCARCHE'].fillna(method='ffill')
        df_loop.loc[:,'TD_1_TAG_DESCARCHE'] = df_loop.loc[:,'TD_1_TAG_DESCARCHE'].fillna(method='ffill')

        df_loop = df_loop.dropna()
    
        df = df.append(df_loop).reset_index(drop=True)
    return(df)
    

df_merge = Interpolate_data(df_merge)

In [29]:
print(df_merge.isnull().mean()*100)

ELEMENT                   0.0
TS                        0.0
TAG_SONDA_PB1             0.0
TAG_SONDA_PB2             0.0
TAG_PRESION_SATURACION    0.0
TAG_TEMP_ASPIRACION       0.0
TAG_RECALENT_VALVULA      0.0
TAG_APERT_VALVULA         0.0
TAG_PETICION_FRIO         0.0
TAG_DESCARCHE             0.0
cycle_number              0.0
SUM_PETICION_FRIO         0.0
SUM_DESCARCHE             0.0
TD_0_TAG_PETICION_FRIO    0.0
TD_1_TAG_PETICION_FRIO    0.0
TD_0_TAG_DESCARCHE        0.0
TD_1_TAG_DESCARCHE        0.0
ID_LOCATION               0.0
dtype: float64


In [30]:
def merging_data(data_telemetria, data_alarmas):
    cycles = data_alarmas['cycle_number'].unique()
    df_timeframe = pd.DataFrame([])
    for i  in range(len(cycles)):
        Alarm_cycle =data_alarmas.loc[data_alarmas['cycle_number']==cycles[i]]
        elemento = Alarm_cycle.iloc[0,0]
        t_max = Alarm_cycle['TS'].max()
        t_min = t_max - relativedelta(days=10)
        df_loop = data_telemetria.loc[data_telemetria['ELEMENT']==elemento]
        df_loop = df_loop.loc[(df_loop['TS'] > t_min) & (df_loop['TS'] <= t_max)]
        if df_loop.shape[0] >3:
            df_loop.at[:,'cycle_number'] = (i+1)
            df_timeframe = df_timeframe.append(df_loop).reset_index(drop=True)

     
    return(df_timeframe)

df_merge = merging_data(df_merge, df_Alarms )

In [31]:
def label_Cycle(data):
    cycles = data['cycle_number'].unique()
    df = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop = data.loc[data['cycle_number']==cycles[i]]
        
        df_loop.loc[:,'cycle_number']=(i+1)
     
        df = df.append(df_loop).reset_index(drop=True)
    return(df)
    
df_merge = label_Cycle(df_merge)

In [32]:
elements_Merge3= df_merge['ELEMENT'].unique()
ciclos_Merge3= df_merge['cycle_number'].unique()
location_Merge3= df_merge['ID_LOCATION'].unique()
print("Count of location in alarms: ",len(location_Merge3))
print("Count of elements in alarms: ",len(elements_Merge3))
print("Count of cycles in alarms: ",len(ciclos_Merge3))
print("this are the excluded elements from the variable with no alarms: " ,list(set(elements_Merge2) - set(elements_Merge3)))
print("Number of elements excuded: " ,len(list(set(elements_Merge2) - set(elements_Merge3))))

Count of location in alarms:  65
Count of elements in alarms:  129
Count of cycles in alarms:  148
this are the excluded elements from the variable with no alarms:  []
Number of elements excuded:  0


<br/><br/>**Import the data of the Central de frio**

In [57]:
query = """

SELECT
	TS,
	ID_LOCATION,
	TAG_POTENCIA_COMP1,
	TAG_POTENCIA_COMP2,
	TAG_POTENCIA_COMP3,
	TAG_POTENCIA_COMP4,
	TAG_POTENCIA_COMP5,
	TAG_POTENCIA_COMP6,
	TAG_SETPOINT_ASP,
	TAG_SETPOINT_COND,
	TAG_SONDA_ASP,
	TAG_SONDA_COND,
	TAG_SONDA_TEMP_EXT,
	TAG_SONDA_TEMP_SUBENF,
FROM
  `table-cloud-labpri-ctrlsenales.DAN_PIC_Europe.PIC_TRACK_VARIABLES_PREP_CENTRALFRIO` A
LEFT JOIN
  `table-cloud-labpri-ctrlsenales.DAN_PIC_Europe.PIC_ELEMENT_LOCATION` B
ON
  (A.ELEMENT = B.ID_ELEMENT);
  
        """
#client = bigquery.Client(location="EU", project="table-cloud-labpri-ctrlsenales")


query_job = client.query(query,location="EU") 

df_central = query_job.to_dataframe()

df_central.head()

,TS,ID_LOCATION,TAG_POTENCIA_COMP1,TAG_POTENCIA_COMP2,TAG_POTENCIA_COMP3,TAG_POTENCIA_COMP4,TAG_POTENCIA_COMP5,TAG_POTENCIA_COMP6,TAG_SETPOINT_ASP,TAG_SETPOINT_COND,TAG_SONDA_ASP,TAG_SONDA_COND,TAG_SONDA_TEMP_EXT,TAG_SONDA_TEMP_SUBENF
0,2019-03-19 09:25:32+00:00,54778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-03-19 09:41:43+00:00,54778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-03-19 09:45:07+00:00,54778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-03-19 09:50:25+00:00,54778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-03-19 09:52:09+00:00,54778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_central['TS'] = df_central['TS'].dt.tz_localize(None)
df_central = df_central.sort_values(['ID_LOCATION','TS'], ascending=[ True, True]).reset_index(drop=True)

In [59]:
#df_central = df_central.loc[(df_central['TAG_CONV_SONDA_ASP'] >-500) ]
#df_central = df_central.loc[(df_central['TAG_CONV_SONDA_COND'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP1'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP2'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP3'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP4'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP5'] >-500) ]
df_central = df_central.loc[(df_central['TAG_POTENCIA_COMP6'] >-500) ]
df_central = df_central.loc[(df_central['TAG_SETPOINT_ASP'] >-500)&(df_central['TAG_SETPOINT_ASP'] <20)  ]
df_central = df_central.loc[(df_central['TAG_SETPOINT_COND'] >-500) ]
df_central = df_central.loc[(df_central['TAG_SONDA_ASP'] >-500) ]
df_central = df_central.loc[(df_central['TAG_SONDA_COND'] >-500) ]
df_central = df_central.loc[(df_central['TAG_SONDA_TEMP_EXT'] >-10) ]
df_central = df_central.loc[(df_central['TAG_SONDA_TEMP_SUBENF'] >-10) ]


In [60]:
df_central =(df_central.set_index('TS').groupby('ID_LOCATION').resample('1min').mean().drop(['ID_LOCATION'], 1).reset_index())

In [61]:
print(round(df_central.isnull().mean()*100,2))

ID_LOCATION               0.00
TS                        0.00
TAG_POTENCIA_COMP1       99.59
TAG_POTENCIA_COMP2       99.59
TAG_POTENCIA_COMP3       99.59
TAG_POTENCIA_COMP4       99.59
TAG_POTENCIA_COMP5       99.59
TAG_POTENCIA_COMP6       99.59
TAG_SETPOINT_ASP         99.59
TAG_SETPOINT_COND        99.59
TAG_SONDA_ASP            99.59
TAG_SONDA_COND           99.59
TAG_SONDA_TEMP_EXT       99.59
TAG_SONDA_TEMP_SUBENF    99.59
dtype: float64


In [62]:
Location_cental= df_central['ID_LOCATION'].unique()
print("Count of Locations in merge2: ",len(Location_cental))
print("this are the excluded elements from the merge: " ,list(set(Location_cental) - set(location_Merge3)))
print("Number of elements with data in central: " ,len(list(set(Location_cental) - set(location_Merge3))))


Count of Locations in merge2:  67
this are the excluded elements from the merge:  [109958, 134346, 12847, 51733, 14231, 53048]
Number of elements with data in central:  6


In [63]:
pd.options.mode.chained_assignment = None

def Interpolate_data(data):
    cycles = data['ID_LOCATION'].unique()
    df = pd.DataFrame([])
    for i, val in enumerate(cycles):
        df_loop = data.loc[data['ID_LOCATION'] == (val)]
        #df_loop.loc[:,'TAG_CONV_SONDA_ASP'] = df_loop.loc[:,'TAG_CONV_SONDA_ASP'].interpolate()
        #df_loop.loc[:,'TAG_CONV_SONDA_COND'] = df_loop.loc[:,'TAG_CONV_SONDA_COND'].interpolate()
        df_loop.loc[:,'TAG_SETPOINT_ASP'] = df_loop.loc[:,'TAG_SETPOINT_ASP'].interpolate()
        df_loop.loc[:,'TAG_SETPOINT_COND'] = df_loop.loc[:,'TAG_SETPOINT_COND'].interpolate()
        df_loop.loc[:,'TAG_SONDA_ASP'] = df_loop.loc[:,'TAG_SONDA_ASP'].interpolate()
        df_loop.loc[:,'TAG_SONDA_COND'] = df_loop.loc[:,'TAG_SONDA_COND'].interpolate()
        df_loop.loc[:,'TAG_SONDA_TEMP_EXT'] = df_loop.loc[:,'TAG_SONDA_TEMP_EXT'].interpolate()
        df_loop.loc[:,'TAG_SONDA_TEMP_SUBENF'] = df_loop.loc[:,'TAG_SONDA_TEMP_SUBENF'].interpolate()

        
        
        #filling the missing values in the binary columns 
        df_loop.loc[:,'TAG_POTENCIA_COMP1'] = df_loop.loc[:,'TAG_POTENCIA_COMP1'].fillna(method='ffill')
        df_loop.loc[:,'TAG_POTENCIA_COMP2'] = df_loop.loc[:,'TAG_POTENCIA_COMP2'].fillna(method='ffill')
        df_loop.loc[:,'TAG_POTENCIA_COMP3'] = df_loop.loc[:,'TAG_POTENCIA_COMP3'].fillna(method='ffill')
        df_loop.loc[:,'TAG_POTENCIA_COMP4'] = df_loop.loc[:,'TAG_POTENCIA_COMP4'].fillna(method='ffill')
        df_loop.loc[:,'TAG_POTENCIA_COMP5'] = df_loop.loc[:,'TAG_POTENCIA_COMP5'].fillna(method='ffill')
        df_loop.loc[:,'TAG_POTENCIA_COMP6'] = df_loop.loc[:,'TAG_POTENCIA_COMP6'].fillna(method='ffill')
        df_loop = df_loop.dropna()
    
        df = df.append(df_loop).reset_index(drop=True)
    return(df)
    

df_central = Interpolate_data(df_central)
df_central

,ID_LOCATION,TS,TAG_POTENCIA_COMP1,TAG_POTENCIA_COMP2,TAG_POTENCIA_COMP3,TAG_POTENCIA_COMP4,TAG_POTENCIA_COMP5,TAG_POTENCIA_COMP6,TAG_SETPOINT_ASP,TAG_SETPOINT_COND,TAG_SONDA_ASP,TAG_SONDA_COND,TAG_SONDA_TEMP_EXT,TAG_SONDA_TEMP_SUBENF
0,7596,2018-12-19 11:35:00,0.0,0.0,100.0,0.0,0.0,0.0,4.190000,7.800000,5.190000,17.799999,35.400002,21.500000
1,7596,2018-12-19 11:36:00,0.0,0.0,100.0,0.0,0.0,0.0,4.193358,7.795523,5.193358,17.795522,35.400002,21.505970
2,7596,2018-12-19 11:37:00,0.0,0.0,100.0,0.0,0.0,0.0,4.196716,7.791045,5.196716,17.791044,35.400002,21.511940
3,7596,2018-12-19 11:38:00,0.0,0.0,100.0,0.0,0.0,0.0,4.200075,7.786567,5.200075,17.786566,35.400002,21.517910
4,7596,2018-12-19 11:39:00,0.0,0.0,100.0,0.0,0.0,0.0,4.203433,7.782090,5.203433,17.782089,35.400002,21.523881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39003847,144625,2020-05-31 23:01:00,0.0,0.0,0.0,0.0,0.0,0.0,1.097333,6.700000,1.082667,6.993333,18.433333,29.819999
39003848,144625,2020-05-31 23:02:00,0.0,0.0,0.0,0.0,0.0,0.0,1.093000,6.700000,1.092000,6.995000,18.425000,29.814999
39003849,144625,2020-05-31 23:03:00,0.0,0.0,0.0,0.0,0.0,0.0,1.088667,6.700000,1.101333,6.996667,18.416666,29.809999
39003850,144625,2020-05-31 23:04:00,0.0,0.0,0.0,0.0,0.0,0.0,1.084333,6.700000,1.110667,6.998333,18.408333,29.804999


In [64]:
print(round(df_central.isnull().mean()*100,2))

ID_LOCATION              0.0
TS                       0.0
TAG_POTENCIA_COMP1       0.0
TAG_POTENCIA_COMP2       0.0
TAG_POTENCIA_COMP3       0.0
TAG_POTENCIA_COMP4       0.0
TAG_POTENCIA_COMP5       0.0
TAG_POTENCIA_COMP6       0.0
TAG_SETPOINT_ASP         0.0
TAG_SETPOINT_COND        0.0
TAG_SONDA_ASP            0.0
TAG_SONDA_COND           0.0
TAG_SONDA_TEMP_EXT       0.0
TAG_SONDA_TEMP_SUBENF    0.0
dtype: float64


In [65]:
Location_cental2= df_central['ID_LOCATION'].unique()
print("Count of Locations in merge2: ",len(Location_cental2))
print("this are the excluded elements from the merge: " ,list(set(Location_cental) - set(Location_cental2)))
print("Number of elements excuded: " ,len(list(set(Location_cental) - set(Location_cental2))))
print("this are the elements with no central data: " ,list(set(location_Merge3) - set(Location_cental2)))


Count of Locations in merge2:  67
this are the excluded elements from the merge:  []
Number of elements excuded:  0
this are the elements with no central data:  [76730, 31301, 91502, 65423]


 [76730, 31301, 91502] this locations represent the following elements 76733 76736 91505 61394 61395 61407

In [66]:
df_merge2= df_merge
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=76733]
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=76736]
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=91505]
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=61394]
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=61395]
df_merge2 =  df_merge2.loc[df_merge2['ELEMENT']!=61407]

In [67]:
elements_merge4= df_merge2['ELEMENT'].unique()
cycles_merge4= df_merge2['cycle_number'].unique()
Location_merge4= df_merge2['ID_LOCATION'].unique()
print("Count of elements in merge3: ",len(elements_merge4))
print("Count of cycles in merge3: ",len(cycles_merge4))
print("Count of Locations in merge3: ",len(Location_merge4))
print("this are the excluded elements from the merge: " ,list(set(elements_Merge3) - set(elements_merge4)))


Count of elements in merge3:  123
Count of cycles in merge3:  142
Count of Locations in merge3:  62
this are the excluded elements from the merge:  [76736, 91505, 61394, 61395, 76733, 61407]




<br/><br/>**Merge the Variables Data with the Central data**

In [68]:

def merge_variables_central(data):
    cycles = data['cycle_number'].unique()
    df_merge_final = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop =  data.loc[data['cycle_number']==cycles[i]]
        locaciones = df_loop['ID_LOCATION'].unique()[0]
        df_loop_central = df_central.loc[df_central['ID_LOCATION']==locaciones]        
        df_loop = df_loop.merge(df_loop_central, on=['ID_LOCATION','TS'],  how='left')
        
        df_merge_final = df_merge_final.append(df_loop).reset_index(drop=True)
    return df_merge_final

df_merge2 = merge_variables_central(df_merge2)
df_merge2


,ELEMENT,TS,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_PETICION_FRIO,TAG_DESCARCHE,...,TAG_POTENCIA_COMP3,TAG_POTENCIA_COMP4,TAG_POTENCIA_COMP5,TAG_POTENCIA_COMP6,TAG_SETPOINT_ASP,TAG_SETPOINT_COND,TAG_SONDA_ASP,TAG_SONDA_COND,TAG_SONDA_TEMP_EXT,TAG_SONDA_TEMP_SUBENF
0,29115,2019-02-27 14:49:00,4.40,1.100000,3.615385,7.500000,9.2000,100.0,1.0,0.0,...,100.0,0.0,0.0,0.0,-9.9,26.170911,3.651113,12.458347,14.479890,23.833389
1,29115,2019-02-27 14:50:00,3.70,0.700000,3.573077,5.100000,7.8000,100.0,1.0,0.0,...,100.0,0.0,0.0,0.0,-9.9,26.170822,3.651111,12.458331,14.479856,23.833324
2,29115,2019-02-27 14:51:00,2.60,0.300000,3.530769,2.700000,6.4000,100.0,1.0,0.0,...,100.0,0.0,0.0,0.0,-9.9,26.170732,3.651109,12.458315,14.479822,23.833259
3,29115,2019-02-27 14:52:00,1.50,-0.100000,3.488462,3.700000,11.4000,100.0,1.0,0.0,...,100.0,0.0,0.0,0.0,-9.9,26.170643,3.651106,12.458299,14.479789,23.833194
4,29115,2019-02-27 14:53:00,1.50,-0.416667,3.446154,4.700000,9.2000,100.0,1.0,0.0,...,100.0,0.0,0.0,0.0,-9.9,26.170553,3.651104,12.458282,14.479755,23.833129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000368,153705,2020-05-02 17:10:00,9.76,6.640000,1.321667,11.105556,18.3375,100.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000369,153705,2020-05-02 17:11:00,9.54,6.160000,1.320000,11.044445,18.5000,100.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000370,153705,2020-05-02 17:12:00,9.32,5.680000,1.318333,10.983333,17.0500,100.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000371,153705,2020-05-02 17:13:00,9.10,5.200000,1.316667,10.922222,15.6000,100.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
print(round(df_merge2.isnull().mean()*100,2))

ELEMENT                   0.00
TS                        0.00
TAG_SONDA_PB1             0.00
TAG_SONDA_PB2             0.00
TAG_PRESION_SATURACION    0.00
TAG_TEMP_ASPIRACION       0.00
TAG_RECALENT_VALVULA      0.00
TAG_APERT_VALVULA         0.00
TAG_PETICION_FRIO         0.00
TAG_DESCARCHE             0.00
cycle_number              0.00
SUM_PETICION_FRIO         0.00
SUM_DESCARCHE             0.00
TD_0_TAG_PETICION_FRIO    0.00
TD_1_TAG_PETICION_FRIO    0.00
TD_0_TAG_DESCARCHE        0.00
TD_1_TAG_DESCARCHE        0.00
ID_LOCATION               0.00
TAG_POTENCIA_COMP1        8.48
TAG_POTENCIA_COMP2        8.48
TAG_POTENCIA_COMP3        8.48
TAG_POTENCIA_COMP4        8.48
TAG_POTENCIA_COMP5        8.48
TAG_POTENCIA_COMP6        8.48
TAG_SETPOINT_ASP          8.48
TAG_SETPOINT_COND         8.48
TAG_SONDA_ASP             8.48
TAG_SONDA_COND            8.48
TAG_SONDA_TEMP_EXT        8.48
TAG_SONDA_TEMP_SUBENF     8.48
dtype: float64


In [73]:
df_merge3 = df_merge2.dropna().reset_index(drop=True)
elements_merge5= df_merge3['ELEMENT'].unique()
cycles_merge5= df_merge3['cycle_number'].unique()
Location_merge5= df_merge3['ID_LOCATION'].unique()
print("Count of elements in merge3: ",len(elements_merge5))
print("Count of cycles in merge3: ",len(cycles_merge5))
print("Count of Locations in merge3: ",len(Location_merge5))
print("this are the excluded elements from the merge: " ,list(set(elements_merge4) - set(elements_merge5)))

Count of elements in merge3:  118
Count of cycles in merge3:  136
Count of Locations in merge3:  60
this are the excluded elements from the merge:  [140806, 153705, 44601, 44602, 44603]


In [74]:
cols = list(df_merge3.columns.values)
df_merge4= df_merge3[[cols[17]]+cols[0:10]+cols[11:17]+cols[18:30]+[cols[10]]]
df_merge4

,ID_LOCATION,ELEMENT,TS,TAG_SONDA_PB1,TAG_SONDA_PB2,TAG_PRESION_SATURACION,TAG_TEMP_ASPIRACION,TAG_RECALENT_VALVULA,TAG_APERT_VALVULA,TAG_PETICION_FRIO,...,TAG_POTENCIA_COMP4,TAG_POTENCIA_COMP5,TAG_POTENCIA_COMP6,TAG_SETPOINT_ASP,TAG_SETPOINT_COND,TAG_SONDA_ASP,TAG_SONDA_COND,TAG_SONDA_TEMP_EXT,TAG_SONDA_TEMP_SUBENF,cycle_number
0,29100,29115,2019-02-27 14:49:00,4.400000,1.100000,3.615385,7.500000,9.200000,100.0,1.0,...,0.0,0.0,0.0,-9.900000,26.170911,3.651113,12.458347,14.479890,23.833389,1
1,29100,29115,2019-02-27 14:50:00,3.700000,0.700000,3.573077,5.100000,7.800000,100.0,1.0,...,0.0,0.0,0.0,-9.900000,26.170822,3.651111,12.458331,14.479856,23.833324,1
2,29100,29115,2019-02-27 14:51:00,2.600000,0.300000,3.530769,2.700000,6.400000,100.0,1.0,...,0.0,0.0,0.0,-9.900000,26.170732,3.651109,12.458315,14.479822,23.833259,1
3,29100,29115,2019-02-27 14:52:00,1.500000,-0.100000,3.488462,3.700000,11.400000,100.0,1.0,...,0.0,0.0,0.0,-9.900000,26.170643,3.651106,12.458299,14.479789,23.833194,1
4,29100,29115,2019-02-27 14:53:00,1.500000,-0.416667,3.446154,4.700000,9.200000,100.0,1.0,...,0.0,0.0,0.0,-9.900000,26.170553,3.651104,12.458282,14.479755,23.833129,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830829,144625,144630,2020-02-22 22:55:00,9.740000,9.360000,1.190000,9.733334,22.433333,100.0,1.0,...,0.0,0.0,0.0,1.106000,6.700000,1.185000,7.160000,13.350000,29.720001,147
1830830,144625,144630,2020-02-22 22:56:00,9.746667,9.353333,1.188333,9.755556,21.400000,100.0,1.0,...,0.0,0.0,0.0,1.110333,6.700000,1.179167,7.153333,13.341666,29.723334,147
1830831,144625,144630,2020-02-22 22:57:00,9.753333,9.346667,1.186667,9.777778,17.700001,100.0,1.0,...,0.0,0.0,0.0,1.114667,6.700000,1.173333,7.146667,13.333333,29.726667,147
1830832,144625,144630,2020-02-22 22:58:00,9.760000,9.340000,1.185000,9.800000,18.350000,100.0,1.0,...,0.0,0.0,0.0,1.119000,6.700000,1.167500,7.140000,13.325000,29.730001,147


In [75]:
#df_merge['unit_number'] = df_merge['ALTA_TEMPERATURA'].map({'Start':0, 'Begin':1}).diff().lt(0).cumsum() + 1
def label_Cycle(data):
    cycles = data['cycle_number'].unique()
    df = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop = data.loc[data['cycle_number']==cycles[i]]
        
        df_loop.loc[:,'cycle_number']=(i+1)
        if df_loop.shape[0] >3:
            
            df = df.append(df_loop).reset_index(drop=True)
    return(df)
    
df_merge5 = label_Cycle(df_merge4)



In [76]:
#df_merge['unit_number'] = df_merge['ALTA_TEMPERATURA'].map({'Start':0, 'Begin':1}).diff().lt(0).cumsum() + 1
def label_Cycle(data):
    cycles = data['cycle_number'].unique()
    df = pd.DataFrame([])
    for i  in range(len(cycles)):
        df_loop = data.loc[data['cycle_number']==cycles[i]]
        
        df_loop.loc[:,'cycle_number']=(i+1)
        df = df.append(df_loop).reset_index(drop=True)
    return(df)
    
df_merge5 = label_Cycle(df_merge5)


In [77]:
coteodeciclos = df_merge5['cycle_number'].unique()
print(coteodeciclos)
print(coteodeciclos.shape)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136]
(136,)


Load the clean Variable data set into Google Cloud Storage

In [78]:
def prepare_train_data(data, factor = 0):
    df = data.copy()
    fd_RUL = df.groupby('cycle_number')['TS'].max().reset_index()
    fd_RUL = pd.DataFrame(fd_RUL)
    fd_RUL.columns = ['cycle_number','max']
    df = df.merge(fd_RUL, on=['cycle_number'], how='left')
    df['RUL'] = ((df['max'] - df['TS']).dt.days*24 + (df['max'] - df['TS']).dt.components.hours).astype(int)
    df.drop(columns=['max'],inplace = True)
    
    return df[df['RUL'] >= factor]

df_merge5 = prepare_train_data(df_merge5)


In [79]:
elements_merge6= df_merge5['ELEMENT'].unique()
cycles_merge6= df_merge5['cycle_number'].unique()
Location_merge6= df_merge5['ID_LOCATION'].unique()
print("Count of elements in merge3: ",len(elements_merge6))
print("Count of cycles in merge3: ",len(cycles_merge6))
print("Count of Locations in merge3: ",len(Location_merge6))
print("this are the excluded elements from the merge: " ,list(set(elements_merge5) - set(elements_merge6)))

Count of elements in merge3:  118
Count of cycles in merge3:  136
Count of Locations in merge3:  60
this are the excluded elements from the merge:  []


In [81]:
client = storage.Client()
bucket = client.get_bucket('table-cloud-lab-ctrlsenales-bucket')
    
bucket.blob('DataSet_128_Var&Central_3.0_1minPeri.csv').upload_from_string(df_merge5.to_csv(), 'text/csv')
print('the dataset in period minutes was load')

the dataset in period minutes was load
